# Unveiling Twitter Sentiments: A Big Data Dive into Twitter Sentiments during the 2020 US Elections

This notebook contains code for the Twitter Sentiment Analysis for Biden DataSet

<!--  -->

### Installing necessary libraries 

In [0]:
#Installing necessary libraries 
%pip install wordninja
%pip install langdetect
%pip install textblob

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


### Initiating Sprak Session

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Twitter_Sentiment_Analysis") \
    .appName("Group_Project") \
    .appName("Biden_V1") \
    .getOrCreate()

sc = spark.sparkContext

In [0]:
sc

Unexpected internal error when monkey patching sc._repr_html_: An error occurred while calling z:com.databricks.backend.daemon.driver.DriverLocal.commandContext. Trace:
py4j.security.Py4JSecurityException: Method public static java.lang.ThreadLocal com.databricks.backend.daemon.driver.DriverLocal.commandContext() is not whitelisted on class class com.databricks.backend.daemon.driver.DriverLocal
	at py4j.security.WhitelistingPy4JSecurityManager.checkCall(WhitelistingPy4JSecurityManager.java:473)
	at py4j.Gateway.invoke(Gateway.java:305)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:199)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:119)
	at java.lang.Thread.run(Thread.java:750)




<SparkContext master=spark://10.139.64.10:7077 appName=Databricks Shell>

### Importing Libraries

In [0]:
#importing necessary libraries 
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark.sql.functions as F
from pyspark.sql.functions import when, col
from pyspark.sql import Row
from pyspark.sql.functions import col, lit
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import regexp_extract, lit, when, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import length, split, size, col, avg, regexp_replace
from pyspark.sql.functions import col, trim
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import split, col, concat_ws
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import wordcloud
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, FloatType
from textblob import TextBlob
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# To ensure that each column's content is fully displayed without being truncated, we have set: 
pd.set_option('display.max_colwidth', None)

### Setting up Spark DataFrame

In [0]:

# Defining my custom schema since we are not doing inferSchema= True, since it will take too much time 
custom_schema_B = StructType([
    StructField("created_at", StringType(), True),
    StructField("tweet_id", StringType(), True),
    StructField("tweet", StringType(), True),
    StructField("likes", StringType(), True),
    StructField("retweet_count", StringType(), True),
    StructField("source", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("user_name", StringType(), True),
    StructField("user_screen_name", StringType(), True),
    StructField("user_description", StringType(), True),
    StructField("user_join_date", StringType(), True),
    StructField("user_followers_count", StringType(), True),
    StructField("user_location", StringType(), True),
    StructField("lat", StringType(), True),
    StructField("long", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("continent", StringType(), True),
    StructField("state", StringType(), True),
    StructField("state_code", StringType(), True),
    StructField("collected_at", StringType(), True),
])


# Reading the CSV file with custom schema
df_b = spark.read.csv("/FileStore/tables/hashtag_joebiden-1.csv", schema = custom_schema_B )
df_b.persist()
#running a count calculation to check how many rows and columns are present in the dataframe
num_rows = df_b.count()
num_columns = len(df_b.columns)
print("DataFrame shape:", (num_rows, num_columns))  

DataFrame shape: (1812042, 21)


In [0]:
#printing the schema to check for the datatypes and everything else 
df_b.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)



In [0]:
df_b= df_b.distinct() #we assumed that just because somebody retweets something doesn't mean they agree with it so we got rid of retweets

### Column details
    created_at: Date and time of tweet creation
    tweet_id: Unique ID of the tweet
    tweet: Full tweet text 
    likes: Number of likes 
    retweet_count: Number of retweets
    source: Utility used to post tweet
    user_id: User ID of tweet creator
    user_name: Username of tweet creator
    user_screen_name: Screen name of tweet creator
    user_description: Description of self by tweet creator
    user_join_date: Join date of tweet creator
    user_followers_count: Followers count on tweet creator
    user_location: Location given on tweet creator's profile
    lat: Latitude parsed from user_location
    long: Longitude parsed from user_location
    city: City parsed from user_location
    country: Country parsed from user_location
    state: State parsed from user_location
    state_code: State code parsed from user_location
    collected_at: Date and time tweet data was mined from twitter*

<!--  -->

#### For Starters we are validating the datetime format of the tweets in the 'created_at' column and then if the format is wrong correcting them and then creating two new columns 'new_created_at' and 'new_tweet' to store the correct formatted tweets

In [0]:

# Assuming valid datetime entries in 'created_at' are in the format "YYYY-MM-DD HH:MM:SS"
datetime_pattern = r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"

# Adjust columns based on datetime validation
df_b = df_b.withColumn(
    "new_created_at",
    when(regexp_extract(col("created_at"), datetime_pattern, 0) != "", col("created_at")).otherwise(lit(None))
)

df_b = df_b.withColumn(
    "new_tweet",
    when(regexp_extract(col("created_at"), datetime_pattern, 0) == "", col("created_at")).otherwise(col("tweet"))
)
df_b.select("new_created_at", "new_tweet").show(truncate=False)


+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|new_created_at     |new_tweet                                                                                                                                                                                           |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NULL               |#ElSolLatino #yobrilloconelsol https://t.co/6FlCBWf1Mi"                                                                                                                                             |
|2020-10-15 00:00:20|@IslandGirlPRV @BradBeauregardJ @MeidasTouch This is how #Biden made his ! #TrumpIsNotAmerica !\n      

### Dropping columns

In [0]:
#now dropping the old 'created_at' and 'tweet' columns since we dont need them anymore 
df_b = df_b.drop('created_at', 'tweet') 
#printing schema to check whether above columns have been dropped or not
df_b.printSchema()


root
 |-- tweet_id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)
 |-- new_created_at: string (nullable = true)
 |-- new_tweet: string (nullable = true)



#### There were empty tweets and they had to be removed since they dont provide any information to us 

<!--  -->

In [0]:
# Filter out rows where 'tweet' is None or empty (including rows that only have whitespace)
df_b = df_b.filter((col("new_tweet").isNotNull()) & (trim(col("new_tweet")) != ""))

In [0]:
df_b.show()

+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------+--------+---------+--------------------+----------+--------------------+-------------------+--------------------+
|            tweet_id|               likes|  retweet_count|              source|             user_id|           user_name|    user_screen_name|    user_description|      user_join_date|user_followers_count|       user_location|    lat|                long|         city| country|continent|               state|state_code|        collected_at|     new_created_at|           new_tweet|
+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----

In [0]:
#calculating how much the data is left after performing basic steps above the count had reduced significantly from 
num_rows = df_b.count()
num_columns = len(df_b.columns)
print("DataFrame shape:", (num_rows, num_columns)) 

DataFrame shape: (1534036, 21)


<!--  -->

# Now cleaning the tweets using the steps below

### Steps:
        1. Convert text to lowercase to standardize it.
        2. Remove contents in square brackets as they usually contain metadata like music info or citations.
        3. Remove URLs to eliminate non-relevant text and potential privacy issues.
        4. Remove any HTML or XML tags that may be present.
        5. Remove new lines to clean up formatting.
        6. Remove words containing numbers to focus only on textual content.
        7. Remove mentions, useful in social media text processing to reduce noise.
        8. Remove hashtags, useful for cleaning tags from social media texts.
        9. Remove retweet artifacts, common in Twitter data.
        10. Replace all non-word characters (except spaces) with spaces to clean up special characters or punctuations.

In [0]:
#creating a udf (user defined function) to remove all the special characters and such as hashtags, emojis, numbers, and lowering all chracters
def clean(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'@[0-9]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

# Registering the function as a UDF
clean_udf = udf(clean, StringType())


# Applying the UDF to the 'new_tweet' column to create 'cleaned_tweet'
df_b = df_b.withColumn("cleaned_tweet", clean_udf(col("new_tweet")))

### At this point basic text pre-processing has been completed after removing retweets, setting correct datetime format, and all special characters and whitespaces

In [0]:
# Filter out rows where 'tweet' is None or empty (including rows that only have whitespace)
df_b = df_b.filter((col("cleaned_tweet").isNotNull()) & (trim(col("cleaned_tweet")) != ""))

<!--  -->

### LANGUAGE  DETECTION IS BEING CONDUCTED HERE TO ONLY INCLUDE THOSE TWEETS THAT ARE IN ENGLISH 

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import wordninja #Probabilistically split concatenated words using NLP based on English Wikipedia uni-gram frequencies.

# Defining a UDF for preprocessing
def split_words(text):
    return ' '.join(wordninja.split(text))

split_words_udf = udf(split_words, StringType())

# Applying the UDF to DataFrame and creating two new columns processed_tweet and cleaned_weet using the split word udf
df_b = df_b.withColumn("processed_tweet", split_words_udf(col("cleaned_tweet")))

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
from langdetect import detect, LangDetectException
def is_english(text):
    # List of words that should always be considered English cause we were seeing some false negatives and thats not what we want
    english_words = {'trump', 'biden', 'election', 'election2020', 'politics'}
    
    # Check if the text is exactly one of the specified words
    if text in english_words:
        return True
    
    try:
        # Returns True if the detected language is English
        return detect(text) == 'en'
    except LangDetectException:
        # If langdetect cannot understand the text, it assumes that the lang is non-English
        return False

# Registering the function as a UDF
is_english_udf = udf(is_english, BooleanType())

In [0]:
# Adding a new column 'is_english' that indicates whether each tweet is in English
df_b = df_b.withColumn("is_english", is_english_udf(col("processed_tweet")))


<!--  -->

#### Counting the avg word length per tweet and some other metrics since these will be required later on to train ML models

In [0]:


# Performing the necessary transformations to create "cleaned_tweet" column
df_b = df_b.withColumn("cleaned_tweet", regexp_replace(col("cleaned_tweet"), "[^a-zA-Z\\s]", ""))

# Calculating the tweet length
df_b = df_b.withColumn("tweet_len", length(col("cleaned_tweet")))

# Calculating the  word count
df_b = df_b.withColumn("word_count", size(split(col("cleaned_tweet"), " ")))

# Calculating the average word length
df_b = df_b.withColumn("avg_word_len", col("tweet_len") / col("word_count"))

# Showing the results
df_b.select("tweet_len", "word_count", "avg_word_len").show()

+---------+----------+------------------+
|tweet_len|word_count|      avg_word_len|
+---------+----------+------------------+
|       14|         2|               7.0|
|        7|         3|2.3333333333333335|
|       47|        12|3.9166666666666665|
|      273|        69|3.9565217391304346|
|       12|         1|              12.0|
|      139|        25|              5.56|
|       18|         1|              18.0|
|       84|        17|4.9411764705882355|
|       40|         6| 6.666666666666667|
|      164|        29| 5.655172413793103|
|       18|         5|               3.6|
|       14|         7|               2.0|
|      135|        15|               9.0|
|       11|         2|               5.5|
|      181|        32|           5.65625|
|       12|         3|               4.0|
|       52|         7| 7.428571428571429|
|       41|         7| 5.857142857142857|
|       69|        16|            4.3125|
|        5|         2|               2.5|
+---------+----------+------------

In [0]:
#checking all columns of the transformed dataframe
column_names = df_b.columns
print(column_names)


['tweet_id', 'likes', 'retweet_count', 'source', 'user_id', 'user_name', 'user_screen_name', 'user_description', 'user_join_date', 'user_followers_count', 'user_location', 'lat', 'long', 'city', 'country', 'continent', 'state', 'state_code', 'collected_at', 'new_created_at', 'new_tweet', 'cleaned_tweet', 'processed_tweet', 'is_english', 'tweet_len', 'word_count', 'avg_word_len']


In [0]:
# Filtering the DataFrame to keep only the tweets that are in English 
df_b = df_b.filter(col("is_english") == True)
df_b.printSchema()





root
 |-- tweet_id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)
 |-- new_created_at: string (nullable = true)
 |-- new_tweet: string (nullable = true)
 |-- cleaned_tweet: string (nullable = true)
 |-- processed_tweet: string (nullable = true)
 |-- is_english: boolean (n

### Removing Stopwords from the cleaned_tweets column and creating a new column known as the filtered_tweet

In [0]:

# Split the processed_tweet column into an array of words
df_b = df_b.withColumn("cleaned_words", split(col("processed_tweet"), " "))

# Create StopWordsRemover instance
remover = StopWordsRemover(inputCol="cleaned_words", outputCol="filtered_words")

# Remove stop words
df_b = remover.transform(df_b)

# Join the filtered words back into a single string
df_b = df_b.withColumn("filtered_tweet", concat_ws(" ", col("filtered_words")))

# Show the DataFrame with filtered tweets
df_b.select("cleaned_tweet", "filtered_tweet").show(truncate=False)



+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cleaned_tweet                                                                                                                                                                                                                                                                              |filtered_tweet                                                                                                                                                                                               

In [0]:
columns_to_drop = ['new_tweet', 'cleaned_tweet', 'processed_tweet','cleaned_words','filtered_words', 'source', 'user_name', 'user_screen_name', 'user_description', 'user_join_date', 'lat', 'long', 'user_id', 'tweet_id', 'new_created_at', 'collected_at','city','country','state','state_code','user_followers_count','retweet_count','likes']
df_b = df_b.alias("df_transformed_3")
df_b = df_b.drop(*columns_to_drop)  

### NOW CACHING THE dataframe "df_transformed_2" since this will be used for sentiment Analysis

In [0]:
df_b.printSchema()

root
 |-- user_location: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- is_english: boolean (nullable = true)
 |-- tweet_len: integer (nullable = true)
 |-- word_count: integer (nullable = false)
 |-- avg_word_len: double (nullable = true)
 |-- filtered_tweet: string (nullable = false)



In [0]:
# Filter the DataFrame to keep only the tweets that are in English 
df_b = df_b.filter(col("is_english") == True)
df_b.printSchema()

root
 |-- user_location: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- is_english: boolean (nullable = true)
 |-- tweet_len: integer (nullable = true)
 |-- word_count: integer (nullable = false)
 |-- avg_word_len: double (nullable = true)
 |-- filtered_tweet: string (nullable = false)



In [0]:
df_b.cache()

DataFrame[user_location: string, continent: string, is_english: boolean, tweet_len: int, word_count: int, avg_word_len: double, filtered_tweet: string]

<!--  -->

# SENTIMENT ANALYSIS

### Using Textblob
### Creating Polarity and catching sentiments using textblob 

In [0]:
from pyspark.sql.types import StringType, FloatType
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_polarity(text):
    return TextBlob(text).sentiment.polarity

def get_analysis(score):
    if score < 0:
        return 'negative'
    elif score == 0:
        return 'neutral'
    else:
        return 'positive'

# Register as UDFs
subjectivity_udf = udf(get_subjectivity, FloatType())
polarity_udf = udf(get_polarity, FloatType())
analysis_udf = udf(get_analysis, StringType())


# Applying UDFs to the DataFrame
df_b = df_b.withColumn('subjectivity', subjectivity_udf(col('filtered_tweet')))
df_b = df_b.withColumn('polarity', polarity_udf(col('filtered_tweet')))
df_b = df_b.withColumn('sentiment', analysis_udf(col('polarity')))



In [0]:
# this was done to write the df back to the csv format since this is df will be used for training ML models
#df_b.unpersist()
#df_b.cache()
#df_b.write.csv("/FileStore/tables/hashtag_joebiden-ML.csv")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4281566180901011>, line 1
----> 1 df_t.show()

NameError: name 'df_t' is not defined

In [0]:
# Assuming df_b is your DataFrame with a 'sentiment' column
sentiment_counts = df_b.groupBy("sentiment").count()

# Show the result
sentiment_counts.show()
#+---------+------+
#|sentiment| count|
#+---------+------+
#| positive|265693|
#|  neutral|385051|
#| negative|110770|
#+---------+------+

+---------+------+
|sentiment| count|
+---------+------+
| positive|265495|
|  neutral|385011|
| negative|110781|
+---------+------+

